This notebook shows how to use the simple model serving functionality found in `tfe.serving`.

# Setup

In [1]:
import tensorflow as tf
import tf_encrypted as tfe

from collections import OrderedDict

2022-11-01 10:11:33.040484: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-01 10:11:33.044530: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-01 10:11:33.044542: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-01 10:11:34.342585: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-01 10:11:34.342610: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to c

# Protocol

We first configure the protocol we will be using, as well as the servers on which we want to run it.

Note that the configuration is saved to file as we will be needing it in the client as well.

In [2]:
players = OrderedDict([
    ('server0', 'localhost:4000'),
    ('server1', 'localhost:4001'),
    ('server2', 'localhost:4002'),
])

config = tfe.RemoteConfig(players)
config.connect_servers()
tfe.set_config(config)
tfe.set_protocol(tfe.protocol.Pond())
config.save('./tfe.config')

2022-11-01 10:11:34.412635: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job tfe -> {0 -> localhost:4000, 1 -> localhost:4001, 2 -> localhost:4002}
2022-11-01 10:11:34.412659: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:44660}
2022-11-01 10:11:34.420067: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job tfe -> {0 -> localhost:4000, 1 -> localhost:4001, 2 -> localhost:4002}
2022-11-01 10:11:34.420085: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:44660}
2022-11-01 10:11:34.420396: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:438] Started server with target: grpc://localhost:44660


## Launching servers

Before actually serving the computation below we need to launch TensorFlow servers in new processes. Run the following in three different terminals. You may have to allow Python to accept incoming connections.

In [3]:
for player_name in players.keys():
    print("python -m tf_encrypted.player --config ./tfe.config {}".format(player_name))

python -m tf_encrypted.player --config ./tfe.config server0
python -m tf_encrypted.player --config ./tfe.config server1
python -m tf_encrypted.player --config ./tfe.config server2


# Computation

We then define the computation we want to run. These will happen on private tensors on the servers defined above.

Note that the only single-tensor computations are currently supported.

In [4]:
input_shape = (5, 5)
output_shape = (5, 5)

def computation(x):
    return x * x

We can then set up a new `tfe.serving.QueueServer` to serve this computation.

In [5]:
server = tfe.serving.QueueServer(
    input_shape=input_shape,
    output_shape=output_shape,
    computation_fn=computation)

# Serving

With all of the above in place we can finally connect to our servers, push our graph to them, and start serving computations.

In [6]:
def step_fn():
    print("Next")

server.run(num_steps=1, step_fn=step_fn)

Next


At this point we switch to the client notebook to run computations.